In [ ]:
# Práctica de Deep Learning

In [2]:
from __future__ import print_function
import keras
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

## Boston Housing Dataset

En este ejercicio vamos a intentar predecir la mediana del precio en un barrio de Boston (a mediados de los 70) a partir de una algunos datos sobre el barrio en la época, como la tasa de criminalidad, el número medio de habitaciones por vivienda en la zona, etc. 

Trabajaremos con el Boston Housing Dataset. Ya hemos utilizado este pequeño conjunto de datos en clase: solamente contamos con 404 instancias para entrenamiento y 102 para testing. Para cada instancia contamos con los siguientes 13 rasgos, acompañados de la siguiente descripción:

1. Per capita crime rate.
2. Proportion of residential land zoned for lots over 25,000 square feet.
3. Proportion of non-retail business acres per town.
4. Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
5. Nitric oxides concentration (parts per 10 million).
6. Average number of rooms per dwelling.
7. Proportion of owner-occupied units built prior to 1940.
8. Weighted distances to five Boston employment centres.
9. Index of accessibility to radial highways.
10. Full-value property-tax rate per $10,000.
11. Pupil-teacher ratio by town.
12. 1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town.
13. % lower status of the population.

Este conjunto de datos ya viene empaquetado como parte del módulo `keras.datasets`. Tas un rápido examen, veremos que cada rasgo tiene un rango de valores diferente. Por eso los normalizaremos todos para que su media sea 0 y desviación estándar 1.

La variable target viene expresada en miles de dólares y los precios están típicamente entre 10 000 y 50 000 dólares (recordemos que los datos son del los años 70).

El siguiente código carga y preprocesa los datos.

In [3]:
from keras.datasets import boston_housing

def get_data():
    (X_train, Y_train), (X_test, Y_test) =  boston_housing.load_data()
    
    mean = X_train.mean(axis=0)
    X_train -= mean
    std = X_train.std(axis=0)
    X_train /= std

    X_test -= mean
    X_test /= std
    
    return X_train, Y_train, X_test, Y_test

## Ejercicio1: Regresión con una FFN (5 ptos.)

Para este problema de regresión, se pide implementar la función `build_model` para que defina y devuelva una FFN con 2 capas ocultas, con 32 neuronas en cada una de ellas. La función de activación en las capas ocultas será ReLU y la red incluirá Dropout con una tasa del 50%.

In [4]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Dropout

def build_model():
    
    # Construimos y compilamos el modelo
    model = Sequential()
    model.add(Dropout(0.5, input_shape=(13,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1,))
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model

El siguiente código entrena la red y evalúa los resultados sobre el conjunto de test.

In [4]:
num_epochs = 100
X_train, Y_train, X_test, Y_test = get_data()
model = build_model()
model.fit(X_train, Y_train,
      epochs=num_epochs, batch_size=1, verbose=1)
test_mse = model.evaluate(X_test, Y_test, verbose=1)
print("Mean Squared Error on test set:", test_mse)

Epoch 1/100
404/404 [==============================] - 0s 757us/step - loss: 405.0840
Epoch 2/100
404/404 [==============================] - 0s 775us/step - loss: 150.9841 0s - loss: 163.94
Epoch 3/100
404/404 [==============================] - 0s 729us/step - loss: 110.3067
Epoch 4/100
404/404 [==============================] - 0s 706us/step - loss: 111.0689
Epoch 5/100
404/404 [==============================] - 0s 734us/step - loss: 115.4042
Epoch 6/100
404/404 [==============================] - 0s 842us/step - loss: 97.4427
Epoch 7/100
404/404 [==============================] - 0s 779us/step - loss: 75.8232
Epoch 8/100
404/404 [==============================] - 0s 793us/step - loss: 102.8489
Epoch 9/100
404/404 [==============================] - 0s 743us/step - loss: 93.5140
Epoch 10/100
404/404 [==============================] - 0s 792us/step - loss: 78.8150
Epoch 11/100
404/404 [==============================] - 0s 852us/step - loss: 91.7407
Epoch 12/100
404/404 [=================

## Ejercicio 2: Selección de hiperparámetros (5 ptos.)

<a href='http://hyperopt.github.io/hyperopt/'>Hyperopt</a> es una biblioteca Python para optimizar hiperparámetros de modelos de Aprendizaje Automático utilizando un algoritmo eficiente de optimización llamado Tree of Parzen Estimators.

<a href='http://maxpumperla.github.io/hyperas/'>Hyperas</a> es una biblioteca que hace todavía más fácil trabajar con Hyperopt y Keras. 

Se pide implementar la función `fit_and_eval_model` en la que crearemos el modelo, lo entrenaremos y lo evaluaremos sobre el conjunto de entrenamiento. Para definir el espacio de hiperparámetros en que Hyperas buscará los valores óptimos, al crear el modelo en lugar de utilizar un valor concreto para el hiperparámetro (por ejemplo, el número de neuronas en una capa) usaremos expresiones como `{{choice([128, 256, 512])}}`. En este caso los valores permitidos para el hiperparámetro serían 128, 256 ó 512.

El espacio de hiperparámetros que queremos definir es:

- Número de capas ocultas: 2 ó 3
- Número de neuronas en cada capa oculta: 32, 64 ó 128.
- Una distribución uniforme en el intervalo [0,1] para cada capa de Drop Out.
- Función de activación en las capas ocultas: relu, sigmoid o tanh.

Échale un vistazo a <a href='http://maxpumperla.github.io/hyperas/'>los ejemplos de la web de Hyperas</a> para que te sirvan de inspiración.

In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation


def fit_and_eval_model(X_train, Y_train, X_test, Y_test):
    num_epochs = 1
    # Completar el codigo y ademas sustuir las 
    # 2 lineas siguientes para asignar valores a model y test_mse 
    model = Sequential()
    model.add(Dropout({{uniform(0, 1)}},input_shape=(13,)))
    model.add(Dense({{choice([32, 64, 128])}}))
    model.add(Activation({{choice(['relu', 'sigmoid', 'tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([32, 64, 128])}}))
    model.add(Activation({{choice(['relu', 'sigmoid', 'tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    
    if conditional({{choice(['four', 'five'])}}) == 'five':
        model.add(Dense({{choice([32, 64, 128])}}))
        model.add(Activation({{choice(['relu', 'sigmoid', 'tanh'])}}))
        model.add(Dropout({{uniform(0, 1)}}))
        
    model.add(Dense(1,))

    model.compile(loss='mean_squared_error', optimizer='adam')
    
    model.fit(X_train, Y_train, epochs=num_epochs, batch_size=1, verbose=1)
    
    test_mse = model.evaluate(X_test, Y_test, verbose=1)
    print("Mean Squared Error on test set:", test_mse)
    
    return {'loss': test_mse, 'status': STATUS_OK, 'model': model}

In [6]:
X_train, Y_train, X_test, Y_test = get_data()

best_run, best_model = optim.minimize(model=fit_and_eval_model,
                          data=get_data,
                          algo=tpe.suggest,
                          max_evals=10,
                          trials=Trials(),
                          notebook_name='Practica_Deep_Learning',
                          eval_space=True)

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import keras
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform, conditional
except:
    pass

try:
    from keras.datasets import boston_housing
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers.core import Dense
except:
    pass

try:
    from keras.layers import Dropout
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dropout': hp.uniform('Dropout', 0, 1),
        'Dense': hp.choice('Dense', [32, 64, 128]),
        'Activation': hp.choice('Activation', ['relu', 'sigmoid', 'tanh']),
        'Dropout_1': hp.uniform('Dropout_1',

In [7]:
best_run

{'Activation': 'relu',
 'Activation_1': 'relu',
 'Activation_2': 'sigmoid',
 'Dense': 32,
 'Dense_1': 64,
 'Dense_2': 32,
 'Dropout': 0.31116431424968705,
 'Dropout_1': 0.20670036025517124,
 'Dropout_2': 0.08078306361277332,
 'Dropout_3': 0.8231073400537193,
 'conditional': 'four'}

In [8]:
print("Evaluation of best performing model:")
print(best_model.evaluate(X_test, Y_test))

Evaluation of best performing model:
102/102 [==============================] - 0s 179us/step
45.3039278517
